In [1]:
import pandas as pd
import pprint
import requests
import time
import csv

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-c50fb4e8-f315-409d-b59b-75cdcf22d58c'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):


    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id, header_option):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    api_status = requests.get(url, headers=request_header).status_code
    if api_status == 200:
        meta_data = requests.get(url, headers=request_header).json()
        if meta_data['info']['queueId'] != 420:
            return False
        else:
            final_result = {}
            final_result['match_id'] = match_id
            final_result['Duration'] = meta_data['info']['gameDuration']
            final_result['Version'] = meta_data['info']['gameVersion']
            participants = meta_data['info']['participants']
            for player in range(len(participants)):
                player_data = get_data_from_dict(participants[player])
                final_result[f'player_{player}'] = player_data
            ban_list_blue_team, ban_list_red_team = [], []
            for blue in meta_data['info']['teams'][0]['bans']:
                ban_list_blue_team.append(blue['championId'])
            for red in meta_data['info']['teams'][1]['bans']:
                ban_list_red_team.append(red['championId'])
            final_result['blue_ban_list'] = ban_list_blue_team
            final_result['red_ban_list'] = ban_list_red_team
            final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
            final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])
            if meta_data['info']['teams'][0]['win'] == False:
                win_team = meta_data['info']['teams'][1]['teamId']
            else:
                win_team = meta_data['info']['teams'][0]['teamId']
            final_result['win_team'] = win_team
            df = pd.json_normalize(final_result)
            df.to_csv(f'MatchId_to_ingame.csv', mode='a', index=False, encoding='CP949', header=header_option)
            return final_result
    else:
        return api_status


def main_run(k):
    df = pd.read_csv('MatchId_data.csv')
    read_data = df['MatchId']
    for i in range(k, len(read_data)):
        cell = i + 2
        try:
            if i == 0:
                header_option = True
            else:
                header_option = False
            rst = main(read_data[i], header_option)
            if rst == 403:
                print("키만료")
                break
            time.sleep(0.9)

        except:
            f = open('error_list.csv', 'a', newline='')
            time.sleep(0.9)
            wr = csv.writer(f)
            wr.writerow([read_data[i]])
            f.close()

        print(f"{cell}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        cell += 1


main_run(0)

2번셀 완료,  k=0 완료,  KR_6493545616
3번셀 완료,  k=1 완료,  KR_6493503693
4번셀 완료,  k=2 완료,  KR_6493452125
5번셀 완료,  k=3 완료,  KR_6493381722
6번셀 완료,  k=4 완료,  KR_6493302123
7번셀 완료,  k=5 완료,  KR_6493216361
8번셀 완료,  k=6 완료,  KR_6493137677
9번셀 완료,  k=7 완료,  KR_6493081490
10번셀 완료,  k=8 완료,  KR_6493029943
11번셀 완료,  k=9 완료,  KR_6493596812
12번셀 완료,  k=10 완료,  KR_6492983234
13번셀 완료,  k=11 완료,  KR_6492943054
14번셀 완료,  k=12 완료,  KR_6492890579
15번셀 완료,  k=13 완료,  KR_6492846914
16번셀 완료,  k=14 완료,  KR_6492806669
17번셀 완료,  k=15 완료,  KR_6492769989
18번셀 완료,  k=16 완료,  KR_6492746708
19번셀 완료,  k=17 완료,  KR_6492718442
20번셀 완료,  k=18 완료,  KR_6492688077
21번셀 완료,  k=19 완료,  KR_6492664387
22번셀 완료,  k=20 완료,  KR_6495145031
23번셀 완료,  k=21 완료,  KR_6495077410
24번셀 완료,  k=22 완료,  KR_6494982112
25번셀 완료,  k=23 완료,  KR_6494919538
26번셀 완료,  k=24 완료,  KR_6494852945
27번셀 완료,  k=25 완료,  KR_6494796516
28번셀 완료,  k=26 완료,  KR_6494320519
29번셀 완료,  k=27 완료,  KR_6494285742
30번셀 완료,  k=28 완료,  KR_6494238766
31번셀 완료,  k=29 완료,  KR_649417278